<a href="https://colab.research.google.com/github/isb-cgc/examples-Python/blob/master/isb_cgc_bam_slicing_with_pysam-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bam slicing in a cloud hosted jupyter notebook.

Welcome to the ‘Query of the Month’. This is part of our collection of new and interesting queries to demonstrate the powerful combination of BigData from the NCI cancer programs like TCGA, and BigQuery from Google.

Please let us know if you have an idea or a suggestion for our next QotM!

Query of the Month is produced by:

David L Gibbs (david.gibbs ( ~ at ~ ) systemsbiology ( ~ dot ~ ) org)
Kawther Abdilleh (kawther.abdilleh ( ~ at ~ ) gdit (~ dot ~) com)
Sheila M Reynolds (sheila.reynolds ( ~ at ~ ) systemsbiology ( ~ dot ~ ) org)

### **In this notebook, using the Pysam package, we demonstrate how to slice bam files stored in GCS buckets.**
### **Here, you'll learn how to: **

1.   How to invoke bash commands within a Jupyter environment. 
2.  How to install packages/programs within a Jupyter environment
3.   How to use available BigQuery tables within ISB-CGC to query and identify Google Cloud Storage bucket locations for BAM files of interest
4.   How to use PySam to slice BAM files
5.   How to save slices in your bucket and retrieve them
6.   Brief example of working with reads

### **Let's authenticate ourselves**

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authorized')

Authorized


### **First, let's prep to install Pysam and the HTSlib**

Pysam is a python wrapper around samtools, and samtools uses the HTSlib (http://www.htslib.org/). So we need to make sure we have the necessary libraries to compile HTSlib and samtools. The compilation is needed to activate the ability to read from google cloud buckets.

In [0]:
import os
os.environ['HTSLIB_CONFIGURE_OPTIONS'] = "--enable-gcs"

### We can invoke bash commands to see what was downloaded into our current working directory. Bash commands can be invoked by putting an exclamation point (!) before the command. 

In [0]:

!ls -lha

total 28K
drwxr-xr-x 1 root root 4.0K Jan 29 23:16 .
drwxr-xr-x 1 root root 4.0K Jan 29 23:14 ..
-rw-r--r-- 1 root root 2.6K Jan 29 23:16 adc.json
drwxr-xr-x 1 root root 4.0K Jan 29 23:16 .config
drwxr-xr-x 1 root root 4.0K Jan 28 17:05 sample_data


In [0]:
!sudo apt-get install autoconf automake make gcc perl zlib1g-dev libbz2-dev liblzma-dev libcurl4-openssl-dev libssl-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
liblzma-dev is already the newest version (5.2.2-1.3).
liblzma-dev set to manually installed.
make is already the newest version (4.1-9.1ubuntu1).
make set to manually installed.
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
gcc is already the newest version (4:7.3.0-3ubuntu2.1).
gcc set to manually installed.
perl is already the newest version (5.26.1-6ubuntu0.3).
perl set to manually installed.
The following additional packages will be installed:
  autotools-dev bzip2-doc libsigsegv2 libssl-doc libssl1.1 m4
Suggested packages:
  autoconf-archive gnu-standards autoconf-doc libtool gettext libcurl4-doc
  libidn11-dev libkrb5-dev libldap2-dev librtmp-dev libssh2-1-dev m4-doc
The following NEW packages will be installed:
  autoconf automake autotools-dev bzip2-doc libbz2-dev libcurl4-openssl-dev
  libsigsegv2 libssl-dev libssl-doc m4
T

In [0]:
!pip3 install pysam -v --force-reinstall --no-binary :all:

# Without forcing the compilation, we get error 
# [Errno 93] could not open alignment file '...': Protocol not supported

Created temporary directory: /tmp/pip-ephem-wheel-cache-grc7x9pg
Created temporary directory: /tmp/pip-req-tracker-_0vdrlxw
Created requirements tracker '/tmp/pip-req-tracker-_0vdrlxw'
Created temporary directory: /tmp/pip-install-4mbemz89
  1 location(s) to search for versions of pysam:
  * https://pypi.org/simple/pysam/
  Getting page https://pypi.org/simple/pysam/
  Looking up "https://pypi.org/simple/pysam/" in the cache
  Request header has "max_age" as 0, cache bypassed
  Starting new HTTPS connection (1): pypi.org:443
  https://pypi.org:443 "GET /simple/pysam/ HTTP/1.1" 200 9072
  Updating cache with response from "https://pypi.org/simple/pysam/"
  Caching due to etag
  Analyzing links from page https://pypi.org/simple/pysam/
    Found link https://files.pythonhosted.org/packages/7f/ce/e9405fcf72096f78dc7d75e8ce2a1cd907e1b4a9af0dbf1fc4dac5770f81/pysam-0.4.tar.gz#sha256=d8d437fadc283be6fc96f932a1417c43f5fb0a7cf267f8a750e819aa2babe1c5 (from https://pypi.org/simple/pysam/), version

In [0]:
import pysam

### **First, we need to set our project. Replace the assignment below with your project ID.**

In [0]:
# First, we need to set our project. Replace the assignment below
# with your project ID.
# project_id = 'isb-cgc-02-0001'
#!gcloud config set project {project_id}

#import os
#os.environ['GCS_OAUTH_TOKEN'] = "gcloud auth application-default print-access-token"


### **Now that we have Pysam installed, let's write an SQL query to locate BAM files in Google Cloud Storage Buckets. **
**In the query below, we are looking to identify the Google Cloud Storage bucket locations for TCGA Ovarian Cancer BAMs obtained via whole genome sequencing (WGS) generated using the SOLiD sequencing system  **


In [0]:
%%bigquery --project isb-cgc-02-0001 df
SELECT * FROM `isb-cgc.TCGA_hg19_data_v0.tcga_metadata_data_hg19_18jul`
where 
data_format = 'BAM' 
AND disease_code = 'OV' 
AND experimental_strategy = "WGS" 
AND platform = 'ABI SOLiD'
LIMIT 5


,file_gdc_id,case_gdc_id,case_barcode,sample_gdc_id,sample_barcode,project_short_name,disease_code,program_name,data_type,data_category,...,type,file_size,data_format,platform,file_name_key,index_file_id,index_file_name_key,index_file_size,access,acl
0,cfb89251-618c-405e-ab84-9d432170a04a,92960f76-0242-4a1c-bf7d-014c2f720219,TCGA-29-1692,a2bc957e-2263-4b0a-b889-70553f59e0e8,TCGA-29-1692-10A,TCGA-OV,OV,TCGA,Aligned reads,Raw sequencing data,...,file,14924511066,BAM,ABI SOLiD,gs://7008814a-277f-4fd4-aa61-flattened/cfb8925...,e3ff985e-47a3-4553-892b-bc7ffb02c776,gs://7008814a-277f-4fd4-aa61-flattened/e3ff985...,7707512,controlled,phs000178
1,18ab4527-5346-437e-829f-27121f372bef,7fd6ab8a-201e-431d-a886-6ab553b6ca36,TCGA-29-1707,0fdb716d-d203-4ac2-9250-d273715055d3,TCGA-29-1707-10A,TCGA-OV,OV,TCGA,Aligned reads,Raw sequencing data,...,file,303193177515,BAM,ABI SOLiD,gs://7008814a-277f-4fd4-aa61-flattened/18ab452...,e0135f4d-9bbb-4395-9e58-75b146c24627,gs://7008814a-277f-4fd4-aa61-flattened/e0135f4...,9044192,controlled,phs000178
2,8006ca66-a708-4296-986b-4c8a91df4cee,a2319490-b85d-4219-a1b0-fa1ec432d5c8,TCGA-29-2414,3a79f24c-a6c3-4b31-9551-37d97c6027ad,TCGA-29-2414-10A,TCGA-OV,OV,TCGA,Aligned reads,Raw sequencing data,...,file,358029548606,BAM,ABI SOLiD,gs://7008814a-277f-4fd4-aa61-flattened/8006ca6...,b26a334f-e553-48e1-9a94-55e556384410,gs://7008814a-277f-4fd4-aa61-flattened/b26a334...,9089040,controlled,phs000178
3,5d04200b-f2c8-4ffc-8540-46ceb1382a9d,7248cd60-be22-44bc-bc58-f644db0940a2,TCGA-13-1489,0276787c-0e07-4452-9eae-54e2778617d7,TCGA-13-1489-10A,TCGA-OV,OV,TCGA,Aligned reads,Raw sequencing data,...,file,13437326688,BAM,ABI SOLiD,gs://7008814a-277f-4fd4-aa61-flattened/5d04200...,c35343b1-74c4-4713-b774-94ae1e60146f,gs://7008814a-277f-4fd4-aa61-flattened/c35343b...,7541408,controlled,phs000178
4,c70ba617-9462-42cd-b043-f66715910a19,7248cd60-be22-44bc-bc58-f644db0940a2,TCGA-13-1489,4b2b6e5e-248c-46f6-938a-95571577e02d,TCGA-13-1489-01A,TCGA-OV,OV,TCGA,Aligned reads,Raw sequencing data,...,file,12072487137,BAM,ABI SOLiD,gs://7008814a-277f-4fd4-aa61-flattened/c70ba61...,10740816-f723-4fe3-b750-a1c0613b9341,gs://7008814a-277f-4fd4-aa61-flattened/1074081...,7508824,controlled,phs000178


### **Now using the following Pysam command,  let's read a bam file from GCS and slice out a section of the bam using the fetch function. For the purposes of the BAM slicing exercise, we will use an open-access CCLE BAM File open-access BAM file.  **[CCLE open access BAM files are stored here](https://console.cloud.google.com/storage/browser/isb-ccle-open/)

In [0]:

samfile = pysam.AlignmentFile('gs://isb-ccle-open/gdc/0a109993-2d5b-4251-bcab-9da4a611f2b1/C836.Calu-3.2.bam', "rb")
for read in samfile.fetch('7', 140453130, 140453135):
  print(read)

samfile.close()

D0MUKACXX120302:5:1103:20853:162833	83	6	140453055	60	76M	6	140453011	76	AAATAGCCTCAATTCTTCCCATCCACAAAATGGATCCAGACAACTGTTCAAACTGATGGGACCCACTCCATCGAGA	array('B', [33, 32, 31, 30, 34, 17, 41, 39, 34, 30, 32, 29, 36, 32, 39, 35, 28, 5, 41, 41, 31, 33, 40, 41, 32, 41, 33, 34, 34, 32, 34, 38, 39, 32, 33, 41, 40, 36, 40, 31, 40, 32, 32, 41, 33, 38, 36, 33, 40, 33, 31, 32, 41, 33, 39, 32, 33, 38, 38, 38, 30, 39, 38, 39, 30, 40, 33, 39, 38, 31, 34, 31, 37, 35, 37, 32])	[('MD', '17A58'), ('PG', 'bwa.19'), ('RG', 'D0MUK.5'), ('AM', 37), ('NM', 1), ('SM', 37), ('MQ', 60), ('OQ', '?DEE?1HD@:D;C8C@8(JJHHIIHJJJJJJJJJJJJJIHIGIJIIIHIJHHJIJJJJIHFIHFDHHHFFFFFCCC'), ('UQ', 5)]
C0FJ4ACXX120306:7:2106:7109:75175	99	6	140453056	60	76M	6	140453125	76	AATAGCCTCAATTCTTACCATCCACAAAATGGATCCAGACAACTGTTCAAACTGATGGGACCCACTCCATCGAGAT	array('B', [29, 34, 30, 29, 35, 37, 36, 35, 36, 30, 25, 28, 33, 24, 35, 32, 30, 23, 38, 32, 32, 31, 37, 29, 37, 33, 35, 35, 33, 31, 39, 35, 32, 31, 39, 37, 33, 40, 26, 38, 33, 25, 35, 3

### The output from the above command is a tab-delimited human readable table of a slice of the BAM file. This table gives us information on reads that mapped to the region that we "extracted" from chromosome 7 between the coordinates of 140453130 and 140453135.


### **Now, let's suppose you would like to save those reads to your own Google cloud storage bucket...**

In [0]:
samfile = pysam.AlignmentFile('gs://isb-ccle-open/gdc/0a109993-2d5b-4251-bcab-9da4a611f2b1/C836.Calu-3.2.bam', "rb")
fetchedreads = pysam.AlignmentFile("test.bam", "wb", template=samfile)
for read in samfile.fetch('7', 140453130, 140453135):
  fetchedreads.write(read)

fetchedreads.close()
samfile.close()

### **Let's see if we saved it?**

In [0]:
!ls -lha

total 5.7M
drwxr-xr-x 1 root root 4.0K Jan 29 23:20 .
drwxr-xr-x 1 root root 4.0K Jan 29 23:14 ..
-rw-r--r-- 1 root root 2.6K Jan 29 23:16 adc.json
-rw-r--r-- 1 root root 5.6M Jan 29 23:20 C836.Calu-3.2.bam.bai
drwxr-xr-x 1 root root 4.0K Jan 29 23:16 .config
drwxr-xr-x 1 root root 4.0K Jan 28 17:05 sample_data
-rw-r--r-- 1 root root  28K Jan 29 23:20 test.bam


In [0]:
#if you don't already have a google cloud storage bucket, you can make one using the following command:
#The mb command creates a new bucket. 
#gsutil mb gs://your_bucket

In [0]:
#to see what's in the bucket..
#!gsutil ls gs://your_bucket/



In [0]:
# then we can copy over the file

!gsutil cp gs://bam_bucket_1/test.bam test_dl.bam

Copying gs://bam_bucket_1/test.bam...
/ [1 files][ 27.5 KiB/ 27.5 KiB]                                                
Operation completed over 1 objects/27.5 KiB.                                     


In [0]:
# and it made it?

!gsutil ls gs://bam_bucket_1/

gs://bam_bucket_1/test.bam
gs://bam_bucket_1/test_2.bam
gs://bam_bucket_1/test_3.bam


### **Now, can we read it back?!?**


In [0]:
newsamfile = pysam.AlignmentFile('gs://bam_bucket_1/test.bam', 'rb')
for r in newsamfile.fetch(until_eof=True):
  print(r)
#  
#  
# No. But maybe soon. #
  

PermissionError: ignored

### Let's move our slice back to this instance.

In [0]:
!gsutil ls gs://bam_bucket_1/

In [0]:
!gsutil cp gs://bam_bucket_1/test.bam test_dl.bam

### Now we're ready to work with our bam-slice!

### Very brief examples of working with reads.

The Alignment file is read as a pysam.AlignedSegment, which is a python class. The methods and class variables can be found here: https://pysam.readthedocs.io/en/latest/api.html#pysam.AlignedSegment

In [0]:
import numpy as np

# first we'll open our bam-slice
dlsamfile = pysam.AlignmentFile('test_dl.bam', 'rb')

# and we'll save the read quality scores in a list
quality = []
for read in dlsamfile:
  quality.append(read.mapping_quality)
  
# then we can compute statistics on them
print("Average quality score")
print(np.mean(quality))

Average quality score
58.861111111111114


In [0]:
# again open our bam-slice
dlsamfile = pysam.AlignmentFile('test_dl.bam', 'rb')

# here, we'll extract the sequences to process
seqs = []
for read in dlsamfile:
  seqs.append(read.query_sequence)
  

# let's count the number of times nucleotide bases are read.
baseCount = dict()
baseCount['A'] = 0
baseCount['C'] = 0
baseCount['G'] = 0
baseCount['T'] = 0
baseCount['N'] = 0
totalCount = 0

for si in seqs:
  for base in si:
    baseCount[base] += 1
    totalCount +=1

for bi in ['A', 'G', 'C', 'T']:
  baseCount[bi] /= totalCount
    
print("Percent bases")
print(baseCount)

Percent bases
{'A': 0.3117690058479532, 'C': 0.24978070175438596, 'G': 0.16564327485380118, 'T': 0.27266081871345027, 'N': 2}
